# 문제 유형4

* 실시간 중계 Tweet이 방문자들의 반응지표들과 관계가 있는지에 대한 분석을 수행하고자 한다. 제공된 데이터셋 정보는 다음과 같다.(2017년 9월 29일부터 10 월 26일까지 개인 계정에서 보낸 313개의 트윗 정보 수집)  

    Tweets2.csv (구분자: “,”(comma), 313 Rows, 13 Columns, UTF-8 Encoding): 

In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

In [2]:
dat=pd.read_csv('data/tweets2.csv')
print('\n 변수목록: ', dat.columns)

print('\n 데이터 모양: ', dat.shape)

print('\n 데이터 모양: ', dat.dtypes)


 변수목록:  Index(['No', 'atConference', 'day', 'impressions', 'retweets', 'likes',
       'userprofileclicks', 'urlClicks', 'hashtagClicks', 'detailExpands',
       'follows', 'mediaViews', 'mediaEngagements'],
      dtype='object')

 데이터 모양:  (313, 13)

 데이터 모양:  No                    int64
atConference           bool
day                  object
impressions           int64
retweets              int64
likes                 int64
userprofileclicks     int64
urlClicks             int64
hashtagClicks         int64
detailExpands         int64
follows               int64
mediaViews            int64
mediaEngagements      int64
dtype: object


In [3]:
# dat['atConference']=dat['atConference'].astype('str')

* 다음의 전처리를 먼저 수행한 후 진행하시오.

단계 1-1. atConference, day, impressions를 제외한 나머지 9개 변수의 합 변수(변수명: totalSum)를 추가하시오.


In [4]:
basetable1=dat.copy()
basetable1['totalSum']=basetable1.drop(columns=['atConference', 'day', 'impressions']).sum(axis=1)

basetable1.head(3)

,No,atConference,day,impressions,retweets,likes,userprofileclicks,urlClicks,hashtagClicks,detailExpands,follows,mediaViews,mediaEngagements,totalSum
0,1,False,2017-10-26,40,0,1,0,0,0,2,0,0,0,4
1,2,False,2017-10-26,876,1,7,2,0,0,3,0,0,0,15
2,3,False,2017-10-25,131,0,2,0,0,0,4,0,0,0,9


* 상기 전처리를 완료한 데이터 프레임(데이터 프레임명: basetable1)으로 다음 분석(문제 1~3)을 수행하시오.

1. (basetable1을 이용) 리-트윗 횟수(retweets)가 0이 아닌 트윗을 대상으로 실시간 중계 여부(atConference)에 따라 좋아요(‘likes’) 클릭 수에 평균 차이가 있는지 적절한 검정을 수행하고 검정 결과의 t-statistic를 기술하시오.

    - 주어진 데이터가 등분산 조건을 만족한다는 가정을 하며, t-statistic은 절대치를 취한 후 소수점 다섯째 자리 이하는 버리고 넷째 자리까지만 기술
    - 정답	4.702598503830562

In [5]:
Q1=basetable1[basetable1.retweets != 0 ][['atConference','likes']]

from scipy.stats import ttest_ind

Q1_out=ttest_ind(Q1[Q1.atConference == True]['likes'], 
                 Q1[Q1.atConference == False]['likes'] )

#Q1_out=ttest_ind(Q1[Q1.atConference == 'True']['likes'], 
#                 Q1[Q1.atConference == 'False']['likes'] )

Q1_ans=abs(Q1_out.statistic)

print('\nQ1 정답: ', Q1_ans)


Q1 정답:  4.702598503830562


* 실시간 중계 여부(atConference)를 예측하는 모델을 생성하기 위해 basetable1에 다음의 전처리를 수행하시오.  

단계 1-2. No, atConference, day를 제외한 10개 변수에 Min-Max Scaler를 적용하시오.


In [6]:
step2=basetable1.copy()

var_list=list(basetable1.columns.drop(['No', 'atConference', 'day', 'totalSum']))

from sklearn.preprocessing import MinMaxScaler

minmax=MinMaxScaler().fit(step2[var_list])

step2.loc[:, var_list]=minmax.transform(step2[var_list])

In [7]:
var_list

['impressions',
 'retweets',
 'likes',
 'userprofileclicks',
 'urlClicks',
 'hashtagClicks',
 'detailExpands',
 'follows',
 'mediaViews',
 'mediaEngagements']

단계 1-3. day변수를 이용하여 요일변수(변수명: weekday)를 추가하시오.

In [8]:
step3=step2.copy()

step3['weekday']=pd.to_datetime(step3.day).dt.day_name(locale='ko_kr')

step3.head(3)

,No,atConference,day,impressions,retweets,likes,userprofileclicks,urlClicks,hashtagClicks,detailExpands,follows,mediaViews,mediaEngagements,totalSum,weekday
0,1,False,2017-10-26,0.001909,0.000000,0.016667,0.000000,0.0,0.0,0.051282,0.0,0.0,0.0,4,목요일
1,2,False,2017-10-26,0.074440,0.029412,0.116667,0.133333,0.0,0.0,0.076923,0.0,0.0,0.0,15,목요일
2,3,False,2017-10-25,0.009804,0.000000,0.033333,0.000000,0.0,0.0,0.102564,0.0,0.0,0.0,9,수요일


단계 1-4. 다음 조건에 따라 train과 test데이터셋을 생성하시오.  

    - train: No가 4의 배수가 아닌 데이터, test: No가 4의 배수인 데이터

In [9]:
train=step3[(step3.No % 4) != 0]
test=step3[(step3.No % 4) == 0]

print('\n train 정보: ', train.shape)
print('\n test 정보: ', test.shape)

print('\n train 정보: ', train.dtypes)


 train 정보:  (235, 15)

 test 정보:  (78, 15)

 train 정보:  No                     int64
atConference            bool
day                   object
impressions          float64
retweets             float64
likes                float64
userprofileclicks    float64
urlClicks            float64
hashtagClicks        float64
detailExpands        float64
follows              float64
mediaViews           float64
mediaEngagements     float64
totalSum               int64
weekday               object
dtype: object


In [10]:
train.head()

,No,atConference,day,impressions,retweets,likes,userprofileclicks,urlClicks,hashtagClicks,detailExpands,follows,mediaViews,mediaEngagements,totalSum,weekday
0,1,False,2017-10-26,0.001909,0.000000,0.016667,0.000000,0.0,0.0,0.051282,0.0,0.000000,0.000000,4,목요일
1,2,False,2017-10-26,0.074440,0.029412,0.116667,0.133333,0.0,0.0,0.076923,0.0,0.000000,0.000000,15,목요일
2,3,False,2017-10-25,0.009804,0.000000,0.033333,0.000000,0.0,0.0,0.102564,0.0,0.000000,0.000000,9,수요일
4,5,False,2017-10-26,0.011192,0.000000,0.050000,0.000000,0.0,0.0,0.051282,0.0,0.114286,0.011429,32,목요일
5,6,False,2017-10-24,0.005813,0.000000,0.016667,0.000000,0.0,0.0,0.025641,0.0,0.000000,0.000000,8,화요일


In [11]:
test.head()

,No,atConference,day,impressions,retweets,likes,userprofileclicks,urlClicks,hashtagClicks,detailExpands,follows,mediaViews,mediaEngagements,totalSum,weekday
3,4,False,2017-10-25,0.124154,0.058824,0.133333,0.533333,0.000000,0.0,0.051282,0.0,0.0,0.0,24,수요일
7,8,False,2017-10-23,1.000000,1.000000,1.000000,1.000000,0.028169,0.0,0.666667,0.0,0.0,0.0,145,월요일
11,12,False,2017-10-21,0.019174,0.000000,0.050000,0.000000,0.000000,0.0,0.025641,0.0,0.0,0.0,16,토요일
15,16,False,2017-10-20,0.156689,0.088235,0.166667,0.333333,0.197183,0.0,0.051282,0.0,0.0,0.0,50,금요일
19,20,False,2017-10-18,0.160420,0.058824,0.183333,0.266667,0.000000,0.0,0.282051,0.0,0.0,0.0,48,수요일


2. 실시간 중계 여부(atConference)를 예측하는 K-NN 분류 모델을 만들고자 한다.

    - 독립 변수(총 10개): Min-Max Scaler를 적용한 변수 사용  
    - 분류 기준: Euclidean 거리를 이용하여 가까운 5개 이웃(Neighbor)의 실시간 중계 여부(atConference) 참조  


도출된 KNN 분류 모델을 통하여 Test 데이터셋의 실시간 중계 여부(atConference) 예측 시, 실제로는 실시간 중계지만(atConference=”TRUE”) 실시간 중계가 아닌 것(predictd=”FALSE”)으로 예측한 데이터의 수는 몇 개인가?


In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report

knn=KNeighborsClassifier(algorithm='auto', n_neighbors=5, leaf_size=30).fit(train[var_list], train['atConference'])

Q2_pred=knn.predict(test[var_list])


Q2_out=confusion_matrix(test['atConference'], Q2_pred)
print(Q2_out)

# Q2_out2=classification_report(test['atConference'], Q2_pred)
# print(Q2_out2)

Q2_ans=Q2_out[1,0]

print('\nQ2 정답: ', Q2_ans)

[[12  7]
 [ 2 57]]

Q2 정답:  2


3. 실시간 중계 여부(atConference)를 예측하는 로지스틱 회귀 분류 모델을 만들고자 한다.  

    - 독립 변수: No, atConference, day, totalSum를 제외한 수치형 변수, weekday의 더미 변수(6개)), train 데이터셋 적용    
    - 모델 가이드 : Inverse of Regularization= 100000, seed=123, Penalty: l2, solver: newton-cg  

도출된 모델에 Test 데이터셋을 적용하여 실시간 중계 여부(atConference)가 “TRUE”인 데이터에 대한 Log Probability TRUE의 합계를 구하시오.( 소수점 다섯째 자리 이하는 버리고 넷째 자리까지만 기술)


In [13]:
train2=pd.get_dummies(train, columns=['weekday'], drop_first=True)
test2=pd.get_dummies(test, columns=['weekday'], drop_first=True)

Q3_var_list=train2.columns.drop(['No', 'atConference', 'day', 'totalSum'])

In [14]:
print(Q3_var_list)

Index(['impressions', 'retweets', 'likes', 'userprofileclicks', 'urlClicks',
       'hashtagClicks', 'detailExpands', 'follows', 'mediaViews',
       'mediaEngagements', 'weekday_목요일', 'weekday_수요일', 'weekday_월요일',
       'weekday_일요일', 'weekday_토요일', 'weekday_화요일'],
      dtype='object')


In [15]:
len(Q3_var_list)

16

In [16]:
from sklearn.linear_model import LogisticRegression

Q3=LogisticRegression(C=100000, random_state=1234, penalty='l2', solver='newton-cg').fit(train2[Q3_var_list], train2['atConference'])
Q3_pred=Q3.predict_log_proba(test2[Q3_var_list])

print(Q3_pred)

[[-7.89392332e-03 -4.84560638e+00]
 [-5.01764592e+00 -6.64210259e-03]
 [ 0.00000000e+00 -4.53285924e+01]
 [ 0.00000000e+00 -5.29387666e+01]
 [-4.02946218e-03 -5.51613642e+00]
 [-1.46600531e-02 -4.22995003e+00]
 [-3.56895992e+00 -2.85899795e-02]
 [-2.76492258e+00 -6.50516932e-02]
 [-4.05567208e+00 -1.74756467e-02]
 [-5.61266008e+00 -3.65802601e-03]
 [-3.86585544e+00 -2.11674558e-02]
 [-3.66362475e+00 -2.59738259e-02]
 [-3.65348245e+00 -2.62421053e-02]
 [-4.71016800e+00 -9.04403918e-03]
 [-3.19056702e+00 -4.20190984e-02]
 [-3.56032511e+00 -2.88415275e-02]
 [-3.14799094e+00 -4.38874233e-02]
 [-3.25185688e+00 -3.94711113e-02]
 [-3.46944250e+00 -3.16293592e-02]
 [-2.31938264e+00 -1.03511420e-01]
 [-5.15360241e+00 -5.79531091e-03]
 [-5.72987822e+00 -3.25275716e-03]
 [-3.51964710e+00 -3.00571054e-02]
 [-5.67167223e+00 -3.44804215e-03]
 [-3.56614025e+00 -2.86718759e-02]
 [-3.93622048e+00 -1.97149270e-02]
 [-2.91891362e+00 -5.55045825e-02]
 [-3.57777349e+00 -2.83355186e-02]
 [-4.10077831e+00 -1

In [17]:
Q3_ans=np.sum(Q3_pred[test2.atConference==True][:,1])

print('\nQ3 정답: ', abs(Q3_ans))


Q3 정답:  2.888403981272016
